<a href="https://colab.research.google.com/github/themysterysolver/FYP-26/blob/main/CODE/MODULE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP-1 : DATASET INGESTION AND CODE GENERATION

- here we are going to make use of 3 dataset MBPP,HumanEval,DS-1000.

##### HumanEval dataset

- got this form hugging face! [HumanEval](https://huggingface.co/datasets/openai/openai_humaneval)
-  164 rows
-  5 features
- `prompt`: The function header and a docstring. This is what you feed the LLM.
- `canonical_solution`: The "perfect" human-written code.
- `test`: A string containing a function (usually named check) that runs several assert statements.
- `entry_point`: The exact name of the function to be tested.
- `task_id`: is the ID of the problem! $Eg:HumanEval/0$

#### STEP 1.1: Loading dataset and understanding dataset!

In [1]:
!pip install datasets
!pip install transformers datasets accelerate torch
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.7 MB/s eta 0:00:00


> **NOTE**: I think I am not using pip install which is being a problem!

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:

from datasets import load_dataset
ds = load_dataset("openai/openai_humaneval")


README.md: 0.00B [00:00, ?B/s]

openai_humaneval/test-00000-of-00001.par(…):   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [4]:
print(ds)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [5]:
df = pd.DataFrame(ds['test'])

In [6]:
print(f'No of rows {df.shape[0]}')
print(f'No of columns {df.shape[1]}')
print(f'columns =>  {df.columns}')

No of rows 164
No of columns 5
columns =>  Index(['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'], dtype='object')


In [7]:
df.head()

,task_id,prompt,canonical_solution,test,entry_point
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",has_close_elements
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",below_zero
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",mean_absolute_deviation


#### STEP 1.2 MODEL LOADING

- we will be using *Qwen2.5-Coder-7B-Instruct*
- Link for Qwen7B => https://huggingface.co/Qwen/Qwen2.5-Coder-7B-Instruct
- Here [Qwen1.5B](https://huggingface.co/Qwen/Qwen2.5-Coder-1.5B) doesn't work because it's poor in generating code.

> *We do not recommend using base language models for conversations. Instead, you can apply post-training, e.g., SFT, RLHF, continued pretraining, etc., or fill in the middle tasks on this model.*

In [ ]:
# !pip uninstall -y bitsandbytes
# !pip install -U bitsandbytes accelerate transformers

> **NOTE:** `-U` is update `-y` is automatic yes!

In [ ]:
# !pip install -U bitsandbytes

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In simple terms, CUDA (Compute Unified Device Architecture) is a platform created by NVIDIA that allows a computer's GPU (Graphics Processing Unit) to perform general-purpose mathematical calculations that are usually handled by the CPU.

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import bitsandbytes as bnb


print(f"BitsAndBytes Version: {bnb.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

ModuleNotFoundError: No module named 'bitsandbytes'

In [ ]:
#CRASHED!!
# model_id = "Qwen/Qwen2.5-Coder-7B-Instruct"

# # 1. Configure 4-bit quantization for memory efficiency
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# # 2. Load Tokenizer and Model
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=bnb_config,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

> **The above model crashed by exceeding Virtual run time by using all the availabel 13GB**

<br>

- System RAM 12.7 GB
- Disk 107.8 GB
- Runtime may last upto 83hrs and 20mins.

##### 1.2.1 VERSION CHECK - UNSLOTH - FAILED

In [ ]:
# # 1. Clear out everything broken
# !pip uninstall -y torch torchvision torchaudio unsloth unsloth-zoo xformers bitsandbytes

# # 2. Install the exact stable versions that Unsloth 2026 requires
# # We target CUDA 12.1 or 12.4 which is standard for T4
# !pip install --no-cache-dir torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
# !pip install --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-cache-dir xformers==0.0.27.post2 trl peft accelerate bitsandbytes

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Found existing installation: unsloth 2026.1.4
Uninstalling unsloth-2026.1.4:
  Successfully uninstalled unsloth-2026.1.4
Found existing installation: unsloth_zoo 2026.1.4
Uninstalling unsloth_zoo-2026.1.4:
  Successfully uninstalled unsloth_zoo-2026.1.4
Found existing installation: xformers 0.0.34
Uninstalling xformers-0.0.34:
  Successfully uninstalled xformers-0.0.34
Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-icr7z0t3/unsloth_2e3d5d2bd17b47de9679d2bf5ff076ae
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-icr7z0t3/unsloth_2e3d5d2bd17b47de9679d2bf5ff076ae
  Resolved https://github.com/unslothai/unsloth.git to commit 4cb7229ac1c346e143524b6f9a6ad544259364d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 193.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 220.3 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2026.1.4-py3-none-any.whl size=412535 sha256=cdf599f65eea81c706acdec25f34a52fed3646975db65e0f4c815a1f4c5579da
  Stored in directory: /tmp/pip-ephem-wheel-cache-8a5mmzwy/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
   ━━━━━

- about the above block :
- It is sued to fix the version erros between the modules.
- We need to restart the session/runtime after updating all the changes.

In [ ]:
import torch
print(f"Is GPU available? {torch.cuda.is_available()}")
# Expected output: True

Is GPU available? True


In [ ]:
# import torch

# # Verification Step: Ensure Inductor is present
# try:
#     import torch._inductor.config
#     print("✅ Torch Inductor found.")
# except AttributeError:
#     print("❌ ERROR: Torch still mismatched. Ensure you restarted the session.")

# import unsloth
# from unsloth import FastLanguageModel

# # We'll use the 3B model - it's much more stable for this hardware
# model_name = "unsloth/Qwen2.5-Coder-3B-Instruct-bnb-4bit"

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = model_name,
#     max_seq_length = 2048,
#     load_in_4bit = True,
# )

# FastLanguageModel.for_inference(model)
# print(f"🚀 Success! {model_name} is ready.")

- Here we are trying to load the modules using unsloth
- hwich has lot's of version compaitablity erros and missing packages for torch such as `int1` error and son.
- we are loading a quantised 4 bit model using bitsandbytes module.

In [ ]:
# import unsloth
# from unsloth import FastLanguageModel
# import torch

# # Define the model you want to use
# model_name = "unsloth/Qwen2.5-Coder-7B-Instruct-bnb-4bit"

# # Load them back into memory
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = model_name,
#     max_seq_length = 2048,
#     load_in_4bit = True,
# )

# # Set to inference mode
# FastLanguageModel.for_inference(model)

# print("✅ Variables 'model' and 'tokenizer' are now defined and ready!")

In [ ]:
# # 3. Prepare the Prompt using the Chat Template
# # This format is required for the "Instruct" model to work properly
# messages = [
#     {"role": "system", "content": "You are an expert Python developer specializing in algorithm optimization."},
#     {"role": "user", "content": "Write a Python function for a non-recursive merge sort and explain its time complexity."}
# ]

# # Apply the template (adds special tokens like <|im_start|> and <|im_end|>)
# prompt = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )

# # 4. Tokenize and Generate
# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# outputs = model.generate(
#     **inputs,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.9,
#     pad_token_id=tokenizer.eos_token_id
# )

# # 5. Get the Response
# response = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)

# print("-" * 30)
# print("MODEL RESPONSE:")
# print(response)

##### 1.2.2 VERSION CHECK - NOT UNSLOTH - 3B

- Trying not the unsloth way!
- Lot's of version errors and memory errors man!
- SO below I am trying to reisntall all packages of torch,tranformers and accelerate anol.
- Restarting the runtime is important for this to work!

In [ ]:
# 1. Uninstall the conflicting versions
!pip uninstall -y torch torchvision torchaudio transformers bitsandbytes accelerate

# 2. Install a stable, matched set (Standard HF stack)
!pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install --no-cache-dir transformers accelerate bitsandbytes

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 330.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 200.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 263.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 194.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 420.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 81.0 MB/s eta 0:00:00


##### 1.2.3 USING 3B (MODEL LOADING)

- Instead of loading `7B` model we use here a `3B` model of QwenCoder.
- This model is quite good in coding accuracy and understanding prompt.
- this is a pre and postrained model unlike 1.5B model

- [Qwen2.5-Coder-3B](https://huggingface.co/Qwen/Qwen2.5-Coder-3B-Instruct)

- Run the below code only after chnaging the runtime environment to `GPU` , *change runtime*

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "Qwen/Qwen2.5-Coder-3B-Instruct"

# 1. Setup 4-bit quantization (Standard Hugging Face way)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically puts it on the GPU
    low_cpu_mem_usage=True # Prevents the System RAM crash
)

print(f"✅ Success! {model_id} loaded using standard transformers.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Success! Qwen/Qwen2.5-Coder-3B-Instruct loaded using standard transformers.


##### 1.2.4 CODE GENERATION

In [ ]:
# Select the prompt from your HumanEval df
task_prompt = df.iloc[0]['prompt']

messages = [
    {"role": "system", "content": "You are a helpful coding assistant. Solve the problem provided in the docstring."},
    {"role": "user", "content": task_prompt}
]

# # Use .to("cpu") or just remove .to("cuda")
# inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cpu")

# with torch.no_grad():
#     # Ensure the model itself is on the cpu
#     model.to("cpu")
#     outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.1)

# response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
# print("--- MODEL GENERATION (CPU) ---")
# print(response)

In [ ]:
# 1. Get the dictionary and move the WHOLE thing to the GPU immediately
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda") # <--- THIS IS THE FIX

with torch.no_grad():
    # Now both the model (on cuda) and the inputs (on cuda) are in the same place
    outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.1)

# Decode and print
response = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
print(response)

To solve this problem, we need to iterate through the list of numbers and check if any two numbers are closer to each other than the specified threshold. Here's how you can implement this:

```python
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    # Sort the list to make it easier to find close elements
    sorted_numbers = sorted(numbers)
    
    # Iterate through the sorted list and check the difference between consecutive elements
    for i in range(len(sorted_numbers) - 1):
        if sorted_numbers[i + 1] - sorted_numbers[i] < threshold:
            return True
    
    # If no such pair is found, return False
    return False

# Example usage
print(has_close_elements([1.0, 2.0, 3.0

In [ ]:
def construct_prompt(docstring_prompt):
    """
    Wraps the HumanEval docstring into a Qwen-style chat template.
    Forces the model to output ONLY the completed code in a python block.
    """
    system_message = (
        "You are an expert Python developer. Your task is to complete the function "
        "provided by the user. Follow the docstring exactly. "
        "Provide your output ONLY as a single Python code block starting with ```python."
    )

    # Structure for the apply_chat_template
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Complete this Python function:\n{docstring_prompt}"}
    ]

    return messages

In [ ]:
# 1. Choose a task (e.g., the 5th problem in your df)
task_id = 4
raw_prompt = df.iloc[task_id]['prompt']

# 2. Construct the message list
formatted_messages = construct_prompt(raw_prompt)

# 3. Tokenize and move to GPU
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

# 4. Generate with specific constraints
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,   # Greedy decoding for consistency
        pad_token_id=tokenizer.eos_token_id
    )

# 5. Extract only the new response
# We use len(inputs['input_ids'][0]) to skip the prompt tokens
generated_ids = outputs[0][len(inputs['input_ids'][0]):]
response = tokenizer.decode(generated_ids, skip_special_tokens=True)

print(f"--- TASK: {df.iloc[task_id]['task_id']} ---")
print(response)

--- TASK: HumanEval/4 ---
```python
from typing import List


def mean_absolute_deviation(numbers: List[float]) -> float:
    """ For a given list of input numbers, calculate Mean Absolute Deviation
    around the mean of this dataset.
    Mean Absolute Deviation is the average absolute difference between each
    element and a centerpoint (mean in this case):
    MAD = average | x - x_mean |
    >>> mean_absolute_deviation([1.0, 2.0, 3.0, 4.0])
    1.0
    """
    if not numbers:
        return 0.0

    mean = sum(numbers) / len(numbers)
    mad = sum(abs(x - mean) for x in numbers) / len(numbers)
    return mad
```


In [ ]:
for i in range(5,10):
    task_id = i
    raw_prompt = df.iloc[task_id]['prompt']
    formatted_messages = construct_prompt(raw_prompt)
    inputs = tokenizer.apply_chat_template(
        formatted_messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to("cuda")

    # 4. Generate with specific constraints
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,   # Greedy decoding for consistency
            pad_token_id=tokenizer.eos_token_id
        )

    # 5. Extract only the new response
    # We use len(inputs['input_ids'][0]) to skip the prompt tokens
    generated_ids = outputs[0][len(inputs['input_ids'][0]):]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)

    print(f"--- TASK: {df.iloc[task_id]['task_id']} ---")
    print(response)

--- TASK: HumanEval/5 ---
```python
from typing import List

def intersperse(numbers: List[int], delimeter: int) -> List[int]:
    """ Insert a number 'delimeter' between every two consecutive elements of input list `numbers'
    >>> intersperse([], 4)
    []
    >>> intersperse([1, 2, 3], 4)
    [1, 4, 2, 4, 3]
    """
    if not numbers:
        return []
    
    result = []
    for i in range(len(numbers)):
        result.append(numbers[i])
        if i < len(numbers) - 1:
            result.append(delimeter)
    return result
```
--- TASK: HumanEval/6 ---
```python
from typing import List

def parse_nested_parens(paren_string: str) -> List[int]:
    """ Input to this function is a string represented multiple groups for nested parentheses separated by spaces.
    For each of the group, output the deepest level of nesting of parentheses.
    E.g. (()()) has maximum two levels of nesting while ((())) has three.

    >>> parse_nested_parens('(()()) ((())) () ((())()())')
    [2, 3, 1,

- model loading and code generation is done above
- Now I wanna do assertion checks for the above code.
- so let's check the **rolling_max** .

#### STEP 1.3 ASSERTION CHECKING

In [ ]:
t = df['test'][9]
print(t)



METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([]) == []
    assert candidate([1, 2, 3, 4]) == [1, 2, 3, 4]
    assert candidate([4, 3, 2, 1]) == [4, 4, 4, 4]
    assert candidate([3, 2, 3, 100, 3]) == [3, 3, 3, 100, 100]




```
METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([]) == []
    assert candidate([1, 2, 3, 4]) == [1, 2, 3, 4]
    assert candidate([4, 3, 2, 1]) == [4, 4, 4, 4]
    assert candidate([3, 2, 3, 100, 3]) == [3, 3, 3, 100, 100]
  ```

- for example 0
```

METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True
    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True
    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False

```

In [ ]:
response = '''--- TASK: HumanEval/9 ---
```python
from typing import List, Tuple

def rolling_max(numbers: List[int]) -> List[int]:
    """ From a given list of integers, generate a list of rolling maximum element found until given moment
    in the sequence.
    >>> rolling_max([1, 2, 3, 2, 3, 4, 2])
    [1, 2, 3, 3, 3, 4, 4]
    """
    if not numbers:
        return []

    max_so_far = numbers[0]
    result = [max_so_far]

    for num in numbers[1:]:
        max_so_far = max(max_so_far, num)
        result.append(max_so_far)

    return result
```'''

##### 1.3.1 PY CODE EXTRACTION USING REGEX

- cleaned the generated code using regex!
- extracting the code using ```python <code> ```

In [ ]:
import re

def extract_python_code(text):
    """
    Extracts code from a markdown-style python block.
    If no block is found, it returns the original text
    (handling cases where the model forgets the tags).
    """
    # Pattern looks for ```python or ``` followed by code and ending with ```
    pattern = r"```(?:python)?\n?(.*?)```"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(1).strip()
    return text.strip()


clean_code = extract_python_code(response)
exec(clean_code)
print(clean_code)

from typing import List, Tuple

def rolling_max(numbers: List[int]) -> List[int]:
    """ From a given list of integers, generate a list of rolling maximum element found until given moment
    in the sequence.
    >>> rolling_max([1, 2, 3, 2, 3, 4, 2])
    [1, 2, 3, 3, 3, 4, 4]
    """
    if not numbers:
        return []

    max_so_far = numbers[0]
    result = [max_so_far]

    for num in numbers[1:]:
        max_so_far = max(max_so_far, num)
        result.append(max_so_far)

    return result


##### 1.3.2 ASSERT CHECKING!


- how to do assertion checks?
- I am going to give the function defention and chekc it.
- Assertion is just a checking statement => ***assert condition, message***

In [ ]:
import sys

def run_test(generated_code, test_script, entry_point):
    """
    generated_code: The clean Python code (from your regex).
    test_script: The 'check' function string from the dataset.
    entry_point: The name of the function (e.g., 'has_close_elements').
    """
    # 1. Combine the generated code and the test script
    # We add a line at the bottom to actually call the test function
    execution_code = f"{generated_code}\n\n{test_script}\ncheck({entry_point})"

    try:
        # 2. Execute the code in a local namespace
        exec_context = {}
        exec(execution_code, exec_context)
        return "Passed"

    except AssertionError as e:
        return "Failed: Logic Hallucination (Assertion Error)"
    except SyntaxError as e:
        return f"Failed: Syntax Hallucination ({e})"
    except Exception as e:
        return f"Failed: {type(e).__name__}: {str(e)}"

run_test(clean_code,t,df['entry_point'][9])

'Passed'

#### STEP 1.4 CSV Dowload HumanEval

- modle loading alone here!

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import sys
import re

In [ ]:
#I ain't doing this and resolving the error in below block!
!pip install datasets
!pip install transformers datasets accelerate torch
!pip install bitsandbytes

In [4]:
# # 1. Uninstall the conflicting versions
# !pip uninstall -y torch torchvision torchaudio transformers bitsandbytes accelerate

# # 2. Install a stable, matched set (Standard HF stack)
# !pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install --no-cache-dir transformers accelerate bitsandbytes

Found existing installation: torch 2.9.0+cpu
Uninstalling torch-2.9.0+cpu:
  Successfully uninstalled torch-2.9.0+cpu
Found existing installation: torchvision 0.24.0+cpu
Uninstalling torchvision-0.24.0+cpu:
  Successfully uninstalled torchvision-0.24.0+cpu
Found existing installation: torchaudio 2.9.0+cpu
Uninstalling torchaudio-2.9.0+cpu:
  Successfully uninstalled torchaudio-2.9.0+cpu
Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 191.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 262.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 M

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 190.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 281.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 235.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0


In [9]:

model_id = "Qwen/Qwen2.5-Coder-3B-Instruct"

# 1. Setup 4-bit quantization (Standard Hugging Face way)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically puts it on the GPU
    low_cpu_mem_usage=True # Prevents the System RAM crash
)

print(f"✅ Success! {model_id} loaded using standard transformers.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Success! Qwen/Qwen2.5-Coder-3B-Instruct loaded using standard transformers.


In [10]:
from datasets import load_dataset
ds = load_dataset("openai/openai_humaneval")
df = pd.DataFrame(ds['test'])

In [11]:
df.to_csv('users.csv', index=False)

In [11]:
def construct_prompt(docstring_prompt):
    """
    Wraps the HumanEval docstring into a Qwen-style chat template.
    Forces the model to output ONLY the completed code in a python block.
    """
    system_message = (
        "You are an expert Python developer. Your task is to complete the function "
        "provided by the user. Follow the docstring exactly. "
        "Provide your output ONLY as a single Python code block starting with ```python."
    )

    # Structure for the apply_chat_template
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Complete this Python function:\n{docstring_prompt}"}
    ]

    return messages
#---------------------------------------------------------------------------------------------------------
def construct_prompt(docstring_prompt):
    """
    Wraps the HumanEval docstring into a Qwen-style chat template.
    Forces the model to output ONLY the completed code in a python block.
    """
    system_message = (
        "You are an expert Python developer. Your task is to complete the function "
        "provided by the user. Follow the docstring exactly. "
        "Provide your output ONLY as a single Python code block starting with ```python."
    )

    # Structure for the apply_chat_template
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Complete this Python function:\n{docstring_prompt}"}
    ]

    return messages

#------------------------------------------------------------------------------------------------------------

def run_test(generated_code, test_script, entry_point):
    """
    generated_code: The clean Python code (from your regex).
    test_script: The 'check' function string from the dataset.
    entry_point: The name of the function (e.g., 'has_close_elements').
    """
    # 1. Combine the generated code and the test script
    # We add a line at the bottom to actually call the test function
    execution_code = f"{generated_code}\n\n{test_script}\ncheck({entry_point})"

    try:
        # 2. Execute the code in a local namespace
        exec_context = {}
        exec(execution_code, exec_context)
        return "Passed"

    except AssertionError as e:
        return "Failed: Logic Hallucination (Assertion Error)"
    except SyntaxError as e:
        return f"Failed: Syntax Hallucination ({e})"
    except Exception as e:
        return f"Failed: {type(e).__name__}: {str(e)}"

#run_test(clean_code,t,df['entry_point'][9])

def extract_python_code(text):
    """
    Extracts code from a markdown-style python block.
    If no block is found, it returns the original text
    (handling cases where the model forgets the tags).
    """
    # Pattern looks for ```python or ``` followed by code and ending with ```
    pattern = r"```(?:python)?\n?(.*?)```"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(1).strip()
    return text.strip()


# clean_code = extract_python_code(response)
# exec(clean_code)
# print(clean_code)

#------------------------------------------------------------------------------------------------------------



In [12]:
import pandas as pd
from google.colab import files
import time

def main_evaluation_driver(start_idx, end_idx, checkpoint_step=10):
    """
    Processes a range of HumanEval problems, generates code,
    tests it, and saves progress.
    """
    # Create a local copy to store results
    current_df = df.iloc[start_idx:end_idx].copy()

    generated_codes = []
    statuses = []

    print(f"🚀 Starting Batch: {start_idx} to {end_idx}")

    for i in range(len(current_df)):
        global_idx = start_idx + i
        row = current_df.iloc[i]

        # 1. Generate
        formatted_messages = construct_prompt(row['prompt'])
        inputs = tokenizer.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        # 2. Extract
        gen_ids = outputs[0][len(inputs['input_ids'][0]):]
        raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
        clean_code = extract_python_code(raw_response)

        # 3. Test
        test_status = run_test(clean_code, row['test'], row['entry_point'])

        # 4. Store
        generated_codes.append(clean_code)
        statuses.append(test_status)

        print(f"✅ Processed {global_idx} ({row['task_id']}): {test_status}\nclean_code:{clean_code}\nstatus:{test_status}\n")

        # Intermediate Checkpoint Saving to CSV (internal to Colab)
        if (i + 1) % checkpoint_step == 0:
            temp_df = current_df.head(len(generated_codes)).copy()
            temp_df['GENERATED_CODE'] = generated_codes
            temp_df['STATUS'] = statuses
            temp_df.to_csv(f"checkpoint_{start_idx}_{global_idx}.csv", index=False)
            print(f"💾 Checkpoint saved at index {global_idx}")

    # 5. Finalize DataFrame
    current_df['GENERATED_CODE'] = generated_codes
    current_df['STATUS'] = statuses

    # Save Final for this batch
    filename = f"humaneval_results_{start_idx}_to_{end_idx}.csv"
    current_df.to_csv(filename, index=False)

    print(f"\n🏁 Batch Finished! File saved as {filename}")
    files.download(filename) # Trigger browser download

    return current_df

# --- EXECUTION ---
# Change these numbers to do it in parts!
# Part 1: main_evaluation_driver(0, 50)
# Part 2: main_evaluation_driver(50, 100)


In [13]:
testing = main_evaluation_driver(0, 3)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚀 Starting Batch: 0 to 3
✅ Processed 0 (HumanEval/0): Passed
clean_code:from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False
status:Passed

['()', '(())', '(()())']
✅ Processed 1 (HumanEval/1): Passed
clean_code:from typing import List

def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each ope

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
results_part1 = main_evaluation_driver(0, 50)

🚀 Starting Batch: 0 to 50
✅ Processed 0 (HumanEval/0): Passed
clean_code:from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False
status:Passed

['()', '(())', '(()())']
✅ Processed 1 (HumanEval/1): Passed
clean_code:from typing import List

def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each op

KeyboardInterrupt: 

In [ ]:
results_part2 = main_evaluation_driver(50, 100)

In [16]:
testing = main_evaluation_driver(100, 165)

🚀 Starting Batch: 100 to 165
All test cases passed!
✅ Processed 100 (HumanEval/100): Passed
clean_code:from typing import List

def make_a_pile(n: int) -> List[int]:
    """
    Given a positive integer n, you have to make a pile of n levels of stones.
    The first level has n stones.
    The number of stones in the next level is:
        - the next odd number if n is odd.
        - the next even number if n is even.
    Return the number of stones in each level in a list, where element at index
    i represents the number of stones in the level (i+1).

    Examples:
    >>> make_a_pile(3)
    [3, 5, 7]
    """
    stones = []
    current_stones = n
    for _ in range(n):
        stones.append(current_stones)
        # Determine the next number of stones based on the current number's parity
        if current_stones % 2 == 0:
            current_stones += 2  # Next even number
        else:
            current_stones += 2  # Next odd number
    return stones

# Check function to verif

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### MBPP dataset

- Most basic python programming [MBPP](https://huggingface.co/datasets/Muennighoff/mbpp)
- It has the folowing features,
  - `source_file` - just the source from where it is took
  - `task_id` - unique ID
  -  `prompt` - the question
  - `code` - the correct solution
  - `test_imports` -
  -  `test_list` - assertion tests

##### Loading dataset

In [4]:
from datasets import load_dataset
dataset = load_dataset("google-research-datasets/mbpp")
sanitized_dataset = load_dataset("google-research-datasets/mbpp", "sanitized")

README.md: 0.00B [00:00, ?B/s]

full/train-00000-of-00001.parquet:   0%|          | 0.00/87.2k [00:00<?, ?B/s]

full/test-00000-of-00001.parquet:   0%|          | 0.00/116k [00:00<?, ?B/s]

full/validation-00000-of-00001.parquet:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

full/prompt-00000-of-00001.parquet:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

...

sanitized/prompt-00000-of-00001.parquet:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/43 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/7 [00:00<?, ? examples/s]

In [5]:
print(sanitized_dataset)

DatasetDict({
    train: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 120
    })
    test: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 257
    })
    validation: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 43
    })
    prompt: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 7
    })
})


In [6]:
mbpp_df = sanitized_dataset['train'].to_pandas()

In [7]:
#print(mbpp_df)
print("MBPP:",mbpp_df.columns)
print("LENGTH:",len(mbpp_df))

MBPP: Index(['source_file', 'task_id', 'prompt', 'code', 'test_imports',
       'test_list'],
      dtype='object')
LENGTH: 120


In [8]:
# print(mbpp_df.iloc[0])
# print(mbpp_df.iloc[1])
mbpp_df.iloc[0]

,0
source_file,Benchmark Questions Verification V2.ipynb
task_id,602
prompt,Write a python function to find the first repe...
code,"def first_repeated_char(str1):\n for index,c ..."
test_imports,[]
test_list,"[assert first_repeated_char(""abcabc"") == ""a"", ..."


In [9]:
def extract_signature(code: str):
    for line in code.splitlines():
        line = line.strip()
        if line.startswith("def "):
            return line
    return None


In [10]:
mbpp_df['function_signature'] = mbpp_df['code'].apply(extract_signature)

In [11]:
mbpp_df[['task_id', 'function_signature']].head()

,task_id,function_signature
0,602,def first_repeated_char(str1):
1,603,def get_ludic(n):
2,604,def reverse_words(s):
3,605,def prime_num(num):
4,606,def radian_degree(degree):


In [12]:
mbpp_df[['task_id','prompt','test_list']]

,task_id,prompt,test_list
0,602,Write a python function to find the first repe...,"[assert first_repeated_char(""abcabc"") == ""a"", ..."
1,603,Write a function to get all lucid numbers smal...,"[assert get_ludic(10) == [1, 2, 3, 5, 7], asse..."
2,604,Write a function to reverse words seperated by...,"[assert reverse_words(""python program"")==(""pro..."
3,605,Write a function to check if the given integer...,"[assert prime_num(13)==True, assert prime_num(..."
4,606,Write a function to convert degrees to radians.,"[assert radian_degree(90)==1.5707963267948966,..."
...,...,...,...
115,805,Write a function that returns the list in a li...,"[assert max_sum_list([[1,2,3], [4,5,6], [10,11..."
116,806,Write a function to find maximum run of upperc...,[assert max_run_uppercase('GeMKSForGERksISBESt...
117,807,Write a python function to find the first odd ...,"[assert first_odd([1,3,5]) == 1, assert first_..."
118,808,Write a function to check if the given tuples ...,"[assert check_K((10, 4, 5, 6, 8), 6) == True, ..."


In [13]:
mbpp_df.columns

Index(['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list',
       'function_signature'],
      dtype='object')

##### Defining functions

In [14]:
def construct_prompt(prompt_text, signature):
    """
    Combines the problem description (prompt) and the exact function
    signature to guide the LLM precisely. we are using MBPP dataset here.
    """
    system_message = (
        "You are an expert Python developer. Your task is to implement a function "
        "based on a description and a specific function signature. "
        "Provide your output ONLY as a single Python code block starting with ```python."
    )

    # We combine the description and the signature so the model knows
    # exactly how to start the code.
    user_content = (
        f"Problem Description:\n{prompt_text}\n\n"
        f"Please implement this exact function:\n{signature}"
    )

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

    return messages
#print(construct_prompt(mbpp_df['prompt'][0],mbpp_df['function_signature'][0]))

In [15]:
def extract_python_code(text):
    """
    Extracts code from a markdown-style python block.
    If no block is found, it returns the original text
    (handling cases where the model forgets the tags).
    """
    # Pattern looks for ```python or ``` followed by code and ending with ```
    pattern = r"```(?:python)?\n?(.*?)```"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(1).strip()
    return text.strip()


#clean_code = extract_python_code(response)

In [16]:
def verify_code(generated_code, test_list):
    namespace = {}
    try:
        exec(generated_code, namespace)
        for test_case in test_list:
            exec(test_case, namespace)
        return "Passed"
    except AssertionError as e:
        return "Failed: Logic Hallucination (Assertion Error)"
    except SyntaxError as e:
        return f"Failed: Syntax Hallucination ({e})"
    except Exception as e:
        return f"Failed: {type(e).__name__}: {str(e)}"

##### PIP INSTALL and IMPORTS

In [1]:
#I ain't doing this and resolving the error in below block!
!pip install datasets
!pip install transformers datasets accelerate torch
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.9 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import sys
import re

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##### MODEL LOADING

In [17]:

model_id = "Qwen/Qwen2.5-Coder-3B-Instruct"

# 1. Setup 4-bit quantization (Standard Hugging Face way)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically puts it on the GPU
    low_cpu_mem_usage=True # Prevents the System RAM crash
)

print(f"✅ Success! {model_id} loaded using standard transformers.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Success! Qwen/Qwen2.5-Coder-3B-Instruct loaded using standard transformers.


##### Trying an example

In [18]:
# 1. Generate
formatted_messages = construct_prompt(mbpp_df['prompt'],mbpp_df['function_signature'])
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )


gen_ids = outputs[0][len(inputs['input_ids'][0]):]
raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
clean_code = extract_python_code(raw_response)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [19]:
print(clean_code)

def first_repeated_char(str1):
    seen = set()
    for char in str1:
        if char in seen:
            return char
        seen.add(char)
    return None

def get_ludic(n):
    ludic = [True] * (n + 1)
    p = 2
    while p * p <= n:
        if ludic[p]:
            for i in range(p * p, n + 1, p):
                ludic[i] = False
        p += 1
    ludic[0] = ludic[1] = False
    return [i for i, x in enumerate(ludic) if x]

def reverse_words(s):
    return ' '.join(word[::-1] for word in s.split())

def prime_num(num):
    if num <= 1:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

def radian_degree(degree):
    import math
    return math.radians(degree)

def max_sum_list(lists):
    return max(sum(lst) for lst in lists)

def max_run_uppercase(test_str):
    max_count = 0
    current_count = 0
    for char in test_str:
        if char.isupper():
            current_count += 1
            max_count

In [20]:
verify_code(clean_code,mbpp_df['test_list'][0])

'Passed'

##### CSV DOWNLOAD

In [23]:
import pandas as pd
from google.colab import files
import time

In [27]:
def mbpp_evaluation_driver(dataframe, start_idx, end_idx, checkpoint_step=10):
    """
    Driver for MBPP dataset using prompt, function_signature, and test_list.
    """
    current_df = dataframe.iloc[start_idx:end_idx].copy()
    generated_codes = []
    statuses = []

    print(f"🚀 Starting MBPP Batch: {start_idx} to {end_idx}")

    for i in range(len(current_df)):
        global_idx = start_idx + i
        row = current_df.iloc[i]

        # 1. Generate using your multi-feature prompt
        # Ensure your construct_prompt accepts (prompt, signature)
        formatted_messages = construct_prompt(row['prompt'], row['function_signature'])

        inputs = tokenizer.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        # 2. Extract
        gen_ids = outputs[0][len(inputs['input_ids'][0]):]
        raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
        clean_code = extract_python_code(raw_response)

        # 3. MBPP Specific Testing
        test_status = verify_code(clean_code,row['test_imports'])

        # 4. Store
        generated_codes.append(clean_code)
        statuses.append(test_status)

        #comment message!
        print(f"✅ [{global_idx}] :Task {row['task_id']}: \nclean_code:\n{clean_code}\nstatus:{test_status}\n")

        # Checkpoint logic
        if (i + 1) % checkpoint_step == 0:
            temp_df = current_df.head(len(generated_codes)).copy()
            temp_df['GENERATED_CODE'] = generated_codes
            temp_df['STATUS'] = statuses
            temp_df.to_csv(f"mbpp_checkpoint_{global_idx}.csv", index=False)

    # 5. Finalize and Download
    current_df['GENERATED_CODE'] = generated_codes
    current_df['STATUS'] = statuses

    filename = f"mbpp_results_{start_idx}_to_{end_idx}.csv"
    current_df.to_csv(filename, index=False)

    print(f"🏁 Batch Finished! Downloading {filename}")
    files.download(filename)

    return current_df

# --- RUN IT ---
# mbpp_results = mbpp_evaluation_driver(mbpp_df, 0, 100)

In [28]:
mbpp_results = mbpp_evaluation_driver(mbpp_df, 0, 3)

🚀 Starting MBPP Batch: 0 to 3
r
None
✅ [0] :Task 602: 
clean_code:
def first_repeated_char(str1):
    seen = set()
    for char in str1:
        if char in seen:
            return char
        seen.add(char)
    return None

# Example usage:
print(first_repeated_char("programming"))  # Output: 'r'
print(first_repeated_char("abcdefg"))     # Output: None
status:Passed

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
✅ [1] :Task 603: 
clean_code:
def get_ludic(n):
    # Initialize the list of ludic numbers
    ludic = [2]
    
    # Start with the first number greater than 2
    candidate = 3
    
    while True:
        # Check if the candidate is less than or equal to n
        if candidate > n:
            break
        
        # Check if the candidate is not divisible by any number in the ludic list
        for num in ludic:
            if candidate % num == 0:
                break
        else:
            # If it's not divisible, add it to the ludic list
            lud

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
mbpp_results = mbpp_evaluation_driver(mbpp_df, 0, 50)

🚀 Starting MBPP Batch: 0 to 50
r
None
✅ [0] :Task 602: 
clean_code:
def first_repeated_char(str1):
    seen = set()
    for char in str1:
        if char in seen:
            return char
        seen.add(char)
    return None

# Example usage:
print(first_repeated_char("programming"))  # Output: 'r'
print(first_repeated_char("abcdefg"))     # Output: None
status:Passed

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
✅ [1] :Task 603: 
clean_code:
def get_ludic(n):
    # Initialize the list of ludic numbers
    ludic = [2]
    
    # Start with the first number greater than 2
    candidate = 3
    
    while True:
        # Check if the candidate is less than or equal to n
        if candidate > n:
            break
        
        # Check if the candidate is not divisible by any number in the ludic list
        for num in ludic:
            if candidate % num == 0:
                break
        else:
            # If it's not divisible, add it to the ludic list
            lu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
mbpp_results = mbpp_evaluation_driver(mbpp_df, 50, 100)

🚀 Starting MBPP Batch: 50 to 100
192
✅ [50] :Task 735: 
clean_code:
def set_middle_bits(n):
    # Get the number of bits in n
    num_bits = n.bit_length()
    
    # Create a mask with all bits set to 1 except the first and last bit
    mask = (1 << (num_bits - 2)) - 1
    
    # Toggle the middle bits by using XOR operation
    result = n ^ mask
    
    return result

# Example usage:
print(set_middle_bits(255))  # Output: 254
status:Passed

2
✅ [51] :Task 736: 
clean_code:
def left_insertion(a, x):
    # Find the index where x should be inserted to maintain sorted order
    left, right = 0, len(a)
    while left < right:
        mid = (left + right) // 2
        if a[mid] < x:
            left = mid + 1
        else:
            right = mid
    return left

# Example usage:
a = [1, 2, 4, 5]
x = 3
print(left_insertion(a, x))  # Output: 2
status:Passed

True
False
✅ [52] :Task 737: 
clean_code:
import re

def check_str(string):
    # Use regex to check if the string starts with a vow

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
mbpp_results = mbpp_evaluation_driver(mbpp_df, 100, 121)

🚀 Starting MBPP Batch: 100 to 121
(1, 2, 3, 'abc')
✅ [100] :Task 788: 
clean_code:
def new_tuple(test_list, test_str):
    # Create a new tuple by combining the elements of the list and the string
    result = tuple(test_list) + (test_str,)
    return result

# Example usage:
test_list = [1, 2, 3]
test_str = "abc"
print(new_tuple(test_list, test_str))  # Output: (1, 2, 3, 'abc')
status:Passed

✅ [101] :Task 790: 
clean_code:
def even_position(nums):
    for i in range(0, len(nums), 2):
        if nums[i] % 2 != 0:
            return False
    return True
status:Passed

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)
✅ [102] :Task 791: 
clean_code:
def remove_nested(test_tup):
    # Initialize an empty list to store non-nested tuples
    result = []
    
    # Iterate over each element in the input tuple
    for item in test_tup:
        # Check if the item is a tuple
        if isinstance(item, tuple):
            # If it's a nested tuple, recursively call the function
            result.extend(re

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- Lemme also try it on the testing data
```
mbpp_df = sanitized_dataset['train'].to_pandas()
```

In [37]:
def mbpp_evaluation_driver_2(dataframe, start_idx, end_idx, checkpoint_step=10):
    """
    Driver for MBPP dataset using prompt, function_signature, and test_list.
    """
    current_df = dataframe.iloc[start_idx:end_idx].copy()
    generated_codes = []
    statuses = []

    print(f"🚀 Starting MBPP Batch: {start_idx} to {end_idx}")

    for i in range(len(current_df)):
        global_idx = start_idx + i
        row = current_df.iloc[i]

        # 1. Generate using your multi-feature prompt
        # Ensure your construct_prompt accepts (prompt, signature)
        formatted_messages = construct_prompt(row['prompt'], row['function_signature'])

        inputs = tokenizer.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        # 2. Extract
        gen_ids = outputs[0][len(inputs['input_ids'][0]):]
        raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
        clean_code = extract_python_code(raw_response)

        # 3. MBPP Specific Testing
        test_status = verify_code(clean_code,row['test_imports'])

        # 4. Store
        generated_codes.append(clean_code)
        statuses.append(test_status)

        #comment message!
        print(f"✅ [{global_idx}] :Task {row['task_id']}: \nclean_code:\n{clean_code}\nstatus:{test_status}\n")

        # Checkpoint logic
        if (i + 1) % checkpoint_step == 0:
            temp_df = current_df.head(len(generated_codes)).copy()
            temp_df['GENERATED_CODE'] = generated_codes
            temp_df['STATUS'] = statuses
            temp_df.to_csv(f"mbpp_checkpoint_test_{global_idx}.csv", index=False)

    # 5. Finalize and Download
    current_df['GENERATED_CODE'] = generated_codes
    current_df['STATUS'] = statuses

    filename = f"mbpp_test_results_{start_idx}_to_{end_idx}.csv"
    current_df.to_csv(filename, index=False)

    print(f"🏁 Batch Finished! Downloading {filename}")
    files.download(filename)

    return current_df

# --- RUN IT ---
# mbpp_results = mbpp_evaluation_driver(mbpp_df, 0, 100)

##### Trying out for test data!

In [32]:
mbpp_df_test = sanitized_dataset['test'].to_pandas()

In [39]:
mbpp_df_test['function_signature'] = mbpp_df_test['code'].apply(extract_signature)

In [40]:
# print(mbpp_df_test)
print(mbpp_df_test.columns)

Index(['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list',
       'function_signature'],
      dtype='object')


In [41]:
mbpp_evaluation_driver_2(mbpp_df_test,0,3)

🚀 Starting MBPP Batch: 0 to 3
hell wold
✅ [0] :Task 11: 
clean_code:
def remove_Occ(s, ch):
    # Find the first occurrence of the character
    first_occurrence = s.find(ch)
    
    # If the character is not found, return the original string
    if first_occurrence == -1:
        return s
    
    # Find the last occurrence of the character
    last_occurrence = s.rfind(ch)
    
    # Remove the first occurrence
    s = s[:first_occurrence] + s[first_occurrence+1:]
    
    # Remove the last occurrence
    s = s[:last_occurrence] + s[last_occurrence+1:]
    
    return s

# Example usage:
print(remove_Occ("hello world", "o"))  # Output: "hell wrld"
status:Passed

[[3, 1, 4], [2, 6, 5], [1, 5, 9]]
✅ [1] :Task 12: 
clean_code:
def sort_matrix(M):
    # Sort the matrix based on the sum of each row
    M.sort(key=sum)
    return M

# Example usage:
matrix = [
    [3, 1, 4],
    [1, 5, 9],
    [2, 6, 5]
]

sorted_matrix = sort_matrix(matrix)
print(sorted_matrix)
status:Passed

✅ [2] :Task

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,source_file,task_id,prompt,code,test_imports,test_list,function_signature,GENERATED_CODE,STATUS
0,Benchmark Questions Verification V2.ipynb,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \n for i in range(len...",[],"[assert remove_Occ(""hello"",""l"") == ""heo"", asse...","def remove_Occ(s,ch):","def remove_Occ(s, ch):\n # Find the first o...",Passed
1,Benchmark Questions Verification V2.ipynb,12,Write a function to sort a given matrix in asc...,"def sort_matrix(M):\n result = sorted(M, ke...",[],"[assert sort_matrix([[1, 2, 3], [2, 4, 5], [1,...",def sort_matrix(M):,def sort_matrix(M):\n # Sort the matrix bas...,Passed
2,Benchmark Questions Verification V2.ipynb,14,Write a python function to find the volume of ...,"def find_Volume(l,b,h) : \n return ((l * b ...",[],"[assert find_Volume(10,8,6) == 240, assert fin...","def find_Volume(l,b,h) :","def find_Volume(l, b, h):\n # Calculate the...",Passed


In [42]:
mbpp_evaluation_driver_2(mbpp_df_test, 0,50)

🚀 Starting MBPP Batch: 0 to 50
hell wold
✅ [0] :Task 11: 
clean_code:
def remove_Occ(s, ch):
    # Find the first occurrence of the character
    first_occurrence = s.find(ch)
    
    # If the character is not found, return the original string
    if first_occurrence == -1:
        return s
    
    # Find the last occurrence of the character
    last_occurrence = s.rfind(ch)
    
    # Remove the first occurrence
    s = s[:first_occurrence] + s[first_occurrence+1:]
    
    # Remove the last occurrence
    s = s[:last_occurrence] + s[last_occurrence+1:]
    
    return s

# Example usage:
print(remove_Occ("hello world", "o"))  # Output: "hell wrld"
status:Passed

[[3, 1, 4], [2, 6, 5], [1, 5, 9]]
✅ [1] :Task 12: 
clean_code:
def sort_matrix(M):
    # Sort the matrix based on the sum of each row
    M.sort(key=sum)
    return M

# Example usage:
matrix = [
    [3, 1, 4],
    [1, 5, 9],
    [2, 6, 5]
]

sorted_matrix = sort_matrix(matrix)
print(sorted_matrix)
status:Passed

✅ [2] :Tas

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,source_file,task_id,prompt,code,test_imports,test_list,function_signature,GENERATED_CODE,STATUS
0,Benchmark Questions Verification V2.ipynb,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \n for i in range(len...",[],"[assert remove_Occ(""hello"",""l"") == ""heo"", asse...","def remove_Occ(s,ch):","def remove_Occ(s, ch):\n # Find the first o...",Passed
1,Benchmark Questions Verification V2.ipynb,12,Write a function to sort a given matrix in asc...,"def sort_matrix(M):\n result = sorted(M, ke...",[],"[assert sort_matrix([[1, 2, 3], [2, 4, 5], [1,...",def sort_matrix(M):,def sort_matrix(M):\n # Sort the matrix bas...,Passed
2,Benchmark Questions Verification V2.ipynb,14,Write a python function to find the volume of ...,"def find_Volume(l,b,h) : \n return ((l * b ...",[],"[assert find_Volume(10,8,6) == 240, assert fin...","def find_Volume(l,b,h) :","def find_Volume(l, b, h):\n # Calculate the...",Passed
3,Benchmark Questions Verification V2.ipynb,16,Write a function to that returns true if the i...,import re\ndef text_lowercase_underscore(text)...,[],"[assert text_lowercase_underscore(""aab_cbbbc"")...",def text_lowercase_underscore(text):,import re\n\ndef text_lowercase_underscore(tex...,Passed
4,Benchmark Questions Verification V2.ipynb,17,Write a function that returns the perimeter of...,def square_perimeter(a):\n perimeter=4*a\n r...,[],"[assert square_perimeter(10)==40, assert squar...",def square_perimeter(a):,def square_perimeter(a):\n return 4 * a,Passed
5,Benchmark Questions Verification V2.ipynb,18,Write a function to remove characters from the...,NO_OF_CHARS = 256\ndef str_to_list(string): \n...,[],"[assert remove_dirty_chars(""probasscurve"", ""pr...",def str_to_list(string):,def str_to_list(string):\n return list(string),Passed
6,Benchmark Questions Verification V2.ipynb,19,Write a function to find whether a given array...,def test_duplicate(arraynums):\n nums_set =...,[],"[assert test_duplicate(([1,2,3,4,5]))==False, ...",def test_duplicate(arraynums):,def test_duplicate(arraynums):\n # Create a...,Passed
7,Benchmark Questions Verification V2.ipynb,20,Write a function to check if the given number ...,def is_woodall(x): \n\tif (x % 2 == 0): \n\t\t...,[],"[assert is_woodall(383) == True, assert is_woo...",def is_woodall(x):,def is_woodall(x):\n # A number x is a Wood...,Passed
8,Mike's Copy of Benchmark Questions Verificatio...,56,Write a python function to check if a given nu...,def rev(num): \n rev_num = 0\n while ...,[],"[assert check(70) == False, assert check(23) =...",def rev(num):,def rev(num):\n # Convert the number to a s...,Passed
9,Mike's Copy of Benchmark Questions Verificatio...,57,Write a python function to find the largest nu...,def find_Max_Num(arr) : \n n = len(arr)\n ...,[],"[assert find_Max_Num([1,2,3]) == 321, assert f...",def find_Max_Num(arr) :,def find_Max_Num(arr):\n # Convert each dig...,Passed


In [43]:
mbpp_evaluation_driver_2(mbpp_df_test, 50,100)

🚀 Starting MBPP Batch: 50 to 100
0
✅ [50] :Task 103: 
clean_code:
def eulerian_num(n, m):
    if m >= n or n == 0:
        return 0
    if m == 0:
        return 1
    if n == 1:
        return 0
    if m == 1:
        return n - 1
    
    # Initialize the base cases
    A = [[0] * (m + 1) for _ in range(n + 1)]
    A[0][0] = 1
    
    # Fill the table using the recurrence relation
    for i in range(1, n + 1):
        A[i][0] = (i - 1) * A[i - 1][1]
        for j in range(1, min(i, m) + 1):
            A[i][j] = (i - 1) * (A[i - 1][j] + A[i - 1][j - 1])
    
    return A[n][m]

# Example usage:
print(eulerian_num(5, 2))  # Output: 15
status:Passed

[['apple', 'banana', 'cherry'], ['cat', 'dog', 'elephant'], ['ant', 'bird', 'fish']]
✅ [51] :Task 104: 
clean_code:
def sort_sublists(input_list):
    # Use the sorted() function to sort each sublist in the input list
    return [sorted(sublist) for sublist in input_list]

# Example usage:
input_list = [["banana", "apple", "cherry"], ["do

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,source_file,task_id,prompt,code,test_imports,test_list,function_signature,GENERATED_CODE,STATUS
50,Mike's Copy of Benchmark Questions Verificatio...,103,Write a function to find the Eulerian number a...,"def eulerian_num(n, m): \n\tif (m >= n or n ==...",[],"[assert eulerian_num(3, 1) == 4, assert euleri...","def eulerian_num(n, m):","def eulerian_num(n, m):\n if m >= n or n ==...",Passed
51,Mike's Copy of Benchmark Questions Verificatio...,104,Write a function to sort each sublist of strin...,def sort_sublists(input_list):\n result = [...,[],"[assert sort_sublists(([""green"", ""orange""], [""...",def sort_sublists(input_list):,def sort_sublists(input_list):\n # Use the ...,Passed
52,Mike's Copy of Benchmark Questions Verificatio...,105,Write a python function to count true booleans...,def count(lst): \n return sum(lst),[],"[assert count([True,False,True]) == 2, assert ...",def count(lst):,def count(lst):\n return sum(1 for item in ...,Passed
53,Mike's Copy of Benchmark Questions Verificatio...,106,Write a function to append the given list to t...,"def add_lists(test_list, test_tup):\n res = t...",[],"[assert add_lists([5, 6, 7], (9, 10)) == (9, 1...","def add_lists(test_list, test_tup):","def add_lists(test_list, test_tup):\n # Con...",Passed
54,Mike's Copy of Benchmark Questions Verificatio...,108,Write a function to merge three lists into a s...,"import heapq\ndef merge_sorted_list(num1,num2,...",[],"[assert merge_sorted_list([25, 24, 15, 4, 5, 2...","def merge_sorted_list(num1,num2,num3):","def merge_sorted_list(num1, num2, num3):\n ...",Passed
55,Mike's Copy of Benchmark Questions Verificatio...,109,Write a python function to find the number of ...,"def odd_Equivalent(s,n): \n count=0\n fo...",[],"[assert odd_Equivalent(""011001"",6) == 3, asser...","def odd_Equivalent(s,n):","def odd_Equivalent(s, n):\n # Convert the b...",Passed
56,Mike's Copy of Benchmark Questions Verificatio...,111,Write a function to find the common elements i...,def common_in_nested_lists(nestedlist):\n r...,[],"[assert set(common_in_nested_lists([[12, 18, 2...",def common_in_nested_lists(nestedlist):,def common_in_nested_lists(nestedlist):\n #...,Passed
57,Mike's Copy of Benchmark Questions Verificatio...,113,Write a function to check if a string represen...,def check_integer(text):\n text = text.strip()...,[],"[assert check_integer(""python"")==False, assert...",def check_integer(text):,def check_integer(text):\n try:\n in...,Passed
58,Mike's Copy of Benchmark Questions Verificatio...,115,Write a function to check whether all dictiona...,def empty_dit(list1):\n empty_dit=all(not d fo...,[],"[assert empty_dit([{},{},{}])==True, assert em...",def empty_dit(list1):,def empty_dit(list1):\n return all(not d fo...,Passed
59,Mike's Copy of Benchmark Questions Verificatio...,116,Write a function to convert a given tuple of p...,def tuple_to_int(nums):\n result = int(''.j...,[],"[assert tuple_to_int((1,2,3))==123, assert tup...",def tuple_to_int(nums):,def tuple_to_int(nums):\n # Convert each nu...,Passed


In [44]:
mbpp_evaluation_driver_2(mbpp_df_test, 100,150)

🚀 Starting MBPP Batch: 100 to 150
✅ [100] :Task 227: 
clean_code:
def min_of_three(a, b, c):
    return min(a, b, c)
status:Passed

False
False
✅ [101] :Task 228: 
clean_code:
def all_Bits_Set_In_The_Given_Range(n, l, r):
    # Create a mask with bits set from l to r (inclusive)
    mask = ((1 << (r - l + 1)) - 1) << (l - 1)
    
    # Check if n & mask equals 0, which means all bits in the range [l, r] are unset
    return (n & mask) == 0

# Example usage:
print(all_Bits_Set_In_The_Given_Range(15, 2, 4))  # Output: True
print(all_Bits_Set_In_The_Given_Range(15, 1, 3))  # Output: False
status:Passed

[-1, -3, 2]
✅ [102] :Task 229: 
clean_code:
def re_arrange_array(arr, n):
    # Separate negative and positive elements
    negatives = [x for x in arr[:n] if x < 0]
    positives = [x for x in arr[:n] if x >= 0]
    
    # Merge them back together, maintaining the original order
    result = negatives + positives
    
    return result

# Example usage:
arr = [-1, 2, -3, 4, -5, 6]
n = 3
p

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,source_file,task_id,prompt,code,test_imports,test_list,function_signature,GENERATED_CODE,STATUS
100,Benchmark Questions Verification V2.ipynb,227,Write a function to find minimum of three numb...,"def min_of_three(a,b,c): \n if (a <= b) a...",[],"[assert min_of_three(10,20,0)==0, assert min_o...","def min_of_three(a,b,c):","def min_of_three(a, b, c):\n return min(a, ...",Passed
101,Benchmark Questions Verification V2.ipynb,228,Write a python function to check whether all t...,"def all_Bits_Set_In_The_Given_Range(n,l,r): \...",[],"[assert all_Bits_Set_In_The_Given_Range(4,1,2)...","def all_Bits_Set_In_The_Given_Range(n,l,r):","def all_Bits_Set_In_The_Given_Range(n, l, r):\...",Passed
102,Benchmark Questions Verification V2.ipynb,229,Write a function that takes in an array and an...,"def re_arrange_array(arr, n):\n j=0\n for i ...",[],"[assert re_arrange_array([-1, 2, -3, 4, 5, 6, ...","def re_arrange_array(arr, n):","def re_arrange_array(arr, n):\n # Separate ...",Passed
103,Benchmark Questions Verification V2.ipynb,230,Write a function that takes in a string and ch...,"def replace_blank(str1,char):\n str2 = str1.re...",[],"[assert replace_blank(""hello people"",'@')==(""h...","def replace_blank(str1,char):","def replace_blank(str1, char):\n return str...",Passed
104,Benchmark Questions Verification V2.ipynb,232,Write a function that takes in a list and an i...,"import heapq\ndef larg_nnum(list1,n):\n larges...",[],"[assert set(larg_nnum([10, 20, 50, 70, 90, 20,...","def larg_nnum(list1,n):","def larg_nnum(list1, n):\n # Sort the list ...",Passed
105,Benchmark Questions Verification V2.ipynb,233,Write a function to find the lateral surface a...,"def lateralsuface_cylinder(r,h):\n lateralsur...",[import math],[assert math.isclose(lateralsuface_cylinder(10...,"def lateralsuface_cylinder(r,h):","import math\n\ndef lateralsurface_cylinder(r, ...",Passed
106,Benchmark Questions Verification V2.ipynb,234,Write a function to find the volume of a cube ...,def volume_cube(l):\n volume = l * l * l\n r...,[],"[assert volume_cube(3)==27, assert volume_cube...",def volume_cube(l):,def volume_cube(l):\n return l ** 3,Passed
107,Benchmark Questions Verification V2.ipynb,235,Write a python function to set all even bits o...,def even_bit_set_number(n): \n count = 0;re...,[],"[assert even_bit_set_number(10) == 10, assert ...",def even_bit_set_number(n):,def even_bit_set_number(n):\n # Initialize ...,Passed
108,Benchmark Questions Verification V2.ipynb,237,Write a function that takes in a list of tuple...,from collections import Counter \ndef check_oc...,[],"[assert check_occurences([(3, 1), (1, 3), (2, ...",def check_occurences(test_list):,def check_occurences(test_list):\n # Initia...,Passed
109,Benchmark Questions Verification V2.ipynb,238,Write a python function to count the number of...,def number_of_substrings(str): \n\tstr_len = l...,[],"[assert number_of_substrings(""abc"") == 6, asse...",def number_of_substrings(str):,def number_of_substrings(s):\n n = len(s)\n...,Passed


In [45]:
mbpp_evaluation_driver_2(mbpp_df_test, 150,200)

🚀 Starting MBPP Batch: 150 to 200
✅ [150] :Task 285: 
clean_code:
```python
import re

def text_match_two_three(text):
    # Use regular expression to find the pattern 'ab{2,3}'
    if re.search(r'ab{2,3}', text):
        return True
    else:
        return False

# Test cases
print(text_match_two_three("abbb"))  # True
print(text_match_two_three("abbbb")) # True
print(text_match_two_three("ab"))     # False
print(text_match_two_three("abbbba")) # True
print(text_match_two_three("abbbbaa"))# True
print(text_match_two_three("abbbbaaa"))# True
print(text_match_two_three("abbbbaaaa"))# True
print(text_match_two_three("abbbbaaaaa"))# True
print(text_match_two_three("abbbbaaaaaa"))# True
print(text_match_two_three("abbbbaaaaaaa"))# True
print(text_match_two_three("abbbbaaaaaaaa"))# True
print(text_match_two_three("abbbbaaaaaaaaaaa"))# True
print(text_match_two_three("abbbbaaaaaaaaaaaa"))# True
print(text_match_two_three("abbbbaaaaaaaaaaaaa"))# True
print(text_match_two_three("abbbbaaaaaaaa

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,source_file,task_id,prompt,code,test_imports,test_list,function_signature,GENERATED_CODE,STATUS
150,Benchmark Questions Verification V2.ipynb,285,Write a function that checks whether a string ...,import re\ndef text_match_two_three(text):\n ...,[],"[assert text_match_two_three(""ac"")==(False), a...",def text_match_two_three(text):,```python\nimport re\n\ndef text_match_two_thr...,Failed: Syntax Hallucination (unterminated str...
151,Benchmark Questions Verification V2.ipynb,286,Write a function to find the largest sum of a ...,"def max_sub_array_sum_repeated(a, n, k): \n\tm...",[],"[assert max_sub_array_sum_repeated([10, 20, -3...","def max_sub_array_sum_repeated(a, n, k):","def max_sub_array_sum_repeated(a, n, k):\n ...",Passed
152,Benchmark Questions Verification V2.ipynb,287,Write a python function takes in an integer n ...,def square_Sum(n): \n return int(2*n*(n+1)...,[],"[assert square_Sum(2) == 20, assert square_Sum...",def square_Sum(n):,def square_Sum(n):\n # Calculate the sum of...,Passed
153,Ellen's Copy of Benchmark Questions Verificati...,290,Write a function to find the list of maximum l...,def max_length(list1):\n max_length = max(l...,[],"[assert max_length([[0], [1, 3], [5, 7], [9, 1...",def max_length(list1):,def max_length(list1):\n if not list1:\n ...,Passed
154,Ellen's Copy of Benchmark Questions Verificati...,291,Write a function to find out the number of way...,"def count_no_of_ways(n, k): \n\tdp = [0] * (n ...",[],"[assert count_no_of_ways(2, 4) == 16, assert c...","def count_no_of_ways(n, k):","def count_no_of_ways(n, k):\n if n == 0:\n ...",Passed
155,Ellen's Copy of Benchmark Questions Verificati...,292,Write a python function to find quotient of tw...,"def find(n,m): \n q = n//m \n return (q)",[],"[assert find(10,3) == 3, assert find(4,2) == 2...","def find(n,m):","def find(n, m):\n return n // m",Passed
156,Ellen's Copy of Benchmark Questions Verificati...,293,Write a function to find the third side of a r...,"import math\ndef otherside_rightangle(w,h):\n ...",[],"[assert otherside_rightangle(7,8)==10.63014581...","def otherside_rightangle(w,h):","import math\n\ndef otherside_rightangle(w, h):...",Passed
157,Ellen's Copy of Benchmark Questions Verificati...,294,Write a function to find the maximum value in ...,def max_val(listval):\n max_val = max(i fo...,[],"[assert max_val(['Python', 3, 2, 4, 5, 'versio...",def max_val(listval):,def max_val(listval):\n # Initialize the ma...,Passed
158,Ellen's Copy of Benchmark Questions Verificati...,295,Write a function to return the sum of all divi...,def sum_div(number):\n divisors = [1]\n ...,[],"[assert sum_div(8)==7, assert sum_div(12)==16,...",def sum_div(number):,def sum_div(number):\n if number <= 0:\n ...,Passed
159,Ellen's Copy of Benchmark Questions Verificati...,296,Write a python function to count inversions in...,def get_Inv_Count(arr): \n inv_count = 0\n ...,[],"[assert get_Inv_Count([1,20,6,4,5]) == 5, asse...",def get_Inv_Count(arr):,def get_Inv_Count(arr):\n def merge_sort_an...,Passed


In [46]:
mbpp_evaluation_driver_2(mbpp_df_test, 200,257)

🚀 Starting MBPP Batch: 200 to 257
✅ [200] :Task 418: 
clean_code:
def Find_Max(lst):
    if not lst:
        return None  # Return None if the list is empty

    max_length = 0
    max_element = None

    for element in lst:
        if len(element) > max_length:
            max_length = len(element)
            max_element = element

    return max_element
status:Passed

44
✅ [201] :Task 419: 
clean_code:
def round_and_sum(list1):
    # Round each number in the list
    rounded_list = [round(num) for num in list1]
    
    # Calculate the sum of the rounded numbers
    total_sum = sum(rounded_list)
    
    # Multiply the sum by the length of the list
    result = total_sum * len(list1)
    
    # Print the result
    print(result)

# Example usage:
round_and_sum([1.2, 2.5, 3.7, 4.1])
status:Passed

288
✅ [202] :Task 420: 
clean_code:
def cube_Sum(n):
    # Calculate the cube sum of the first n even natural numbers
    return sum((2 * i) ** 3 for i in range(1, n + 1))

# Example usage:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,source_file,task_id,prompt,code,test_imports,test_list,function_signature,GENERATED_CODE,STATUS
200,charlessutton@: Benchmark Questions Verificati...,418,Write a python function to find the element of...,def Find_Max(lst): \n maxList = max((x) for...,[],"[assert Find_Max([['A'],['A','B'],['A','B','C'...",def Find_Max(lst):,def Find_Max(lst):\n if not lst:\n r...,Passed
201,charlessutton@: Benchmark Questions Verificati...,419,Write a function to round every number of a gi...,def round_and_sum(list1):\n lenght=len(list1)...,[],"[assert round_and_sum([22.4, 4.0, -16.22, -9.1...",def round_and_sum(list1):,def round_and_sum(list1):\n # Round each nu...,Passed
202,charlessutton@: Benchmark Questions Verificati...,420,Write a python function to find the cube sum o...,def cube_Sum(n): \n sum = 0\n for i in r...,[],"[assert cube_Sum(2) == 72, assert cube_Sum(3) ...",def cube_Sum(n):,def cube_Sum(n):\n # Calculate the cube sum...,Passed
203,charlessutton@: Benchmark Questions Verificati...,421,Write a function to concatenate each element o...,def concatenate_tuple(test_tup):\n delim = ...,[],"[assert concatenate_tuple((""ID"", ""is"", 4, ""UTS...",def concatenate_tuple(test_tup):,def concatenate_tuple(test_tup):\n # Initia...,Passed
204,charlessutton@: Benchmark Questions Verificati...,422,Write a python function to find the average of...,def find_Average_Of_Cube(n): \n sum = 0\n ...,[],"[assert find_Average_Of_Cube(2) == 4.5, assert...",def find_Average_Of_Cube(n):,def find_Average_Of_Cube(n):\n # Calculate ...,Passed
205,charlessutton@: Benchmark Questions Verificati...,424,Write a function to extract only the rear inde...,def extract_rear(test_tuple):\n res = list(su...,[],"[assert extract_rear(('Mers', 'for', 'Vers') )...",def extract_rear(test_tuple):,def extract_rear(test_tuple):\n # Extractin...,Passed
206,charlessutton@: Benchmark Questions Verificati...,425,Write a function to count the number of sublis...,"def count_element_in_list(list1, x): \n ctr...",[],"[assert count_element_in_list([[1, 3], [5, 7],...","def count_element_in_list(list1, x):","def count_element_in_list(list1, x):\n coun...",Passed
207,charlessutton@: Benchmark Questions Verificati...,426,Write a function to filter odd numbers.,def filter_oddnumbers(nums):\n odd_nums = list...,[],"[assert filter_oddnumbers([1, 2, 3, 4, 5, 6, 7...",def filter_oddnumbers(nums):,def filter_oddnumbers(nums):\n return [num ...,Passed
208,charlessutton@: Benchmark Questions Verificati...,427,Write a function to convert a date of yyyy-mm-...,import re\ndef change_date_format(dt):\n ...,[],"[assert change_date_format(""2026-01-02"") == '0...",def change_date_format(dt):,from datetime import datetime\n\ndef change_da...,Passed
209,charlessutton@: Benchmark Questions Verificati...,428,Write a function to sort the given array by us...,def shell_sort(my_list):\n gap = len(my_lis...,[],"[assert shell_sort([12, 23, 4, 5, 3, 2, 12, 81...",def shell_sort(my_list):,def shell_sort(my_list):\n n = len(my_list)...,Passed


In [47]:
# This tells the notebook not to save the widget state in the metadata
import ipywidgets as widgets
widgets.Widget.close_all()